# The Julia Package manager

A key strength of Julia is that it is easy to reuse other peoples code. To facilitate this, Julia includes a package manager, named Pkg. Here are some of the things it does:

- adding and removing packages
- create project environments using Project.toml
- allow fully reproducible projects using Manifest.toml
- specifying compatibility with dependencies
- facilitate developing packages

## Links
- Pkg documentation - https://julialang.github.io/Pkg.jl/v1/
- BinaryBuilder, package binary dependencies - https://binarybuilder.org/
- Package compiler, bundle relocatable projects - https://julialang.github.io/PackageCompiler.jl/dev/
- General package registry - https://github.com/JuliaRegistries/General/
- JuliaHub (beta), browse available packages - https://juliahub.com/
- Julia Packages, browse available packages - https://juliapackages.com/

## Adding and removing packages

The most convenient way to use the package manager is to use the Pkg REPL-mode. To enter the Pkg REPL-mode in an interactive session, type `]` in the prompt. The prompt indicator will go from `julia>` to `pkg>`. In a Jupyter notebook, just start the cell with `]` for the same effect. Remember that with `]?` you can always get help on the Pkg REPL-mode, and `]?add` for specific help for `add`, or any other command.

In [4]:
]?

  Welcome to the Pkg REPL-mode. To return to the julia> prompt, either press
  backspace when the input line is empty or press Ctrl+C.

  Synopsis

  pkg> cmd [opts] [args]

  Multiple commands can be given on the same line by interleaving a ; between
  the commands. Some commands have an alias, indicated below.

  Commands

  activate: set the primary environment the package manager manipulates

  add: add packages to project

  build: run the build script for packages

  develop, dev: clone the full package repo locally for development

  free: undoes a pin, develop, or stops tracking a repo

  gc: garbage collect packages not used for a significant time

  generate: generate files for a new project

  help, ?: show this message

  instantiate: downloads all the dependencies for the project

  pin: pins the version of packages

  precompile: precompile all the project dependencies

  redo: redo the latest change to the active project

  remove, rm: remove packages from project or man

You can add packages to your current environment using `]add Example`. It is also possible to add multiple packages at the same time, or specify a desired version. By default it will add the most recent version that is compatible with the packages that are already installed.

In [23]:
]add Example UnicodePlots

  Resolving package versions...
No Changes to `C:\Users\visser_mn\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\visser_mn\.julia\environments\v1.5\Manifest.toml`


When packages are added to an environment, they can directly be used. The first time a (new version of a) package is loaded, it will precompile the code, to make it faster to load in the future. If you want, you can also force this to happen for all packages using `]precompile`.

In [11]:
using UnicodePlots

┌ Info: Precompiling UnicodePlots [b8865327-cd53-5732-bb35-84acbb429228]
└ @ Base loading.jl:1278


[UnicodePlots](https://github.com/Evizero/UnicodePlots.jl) is a fun package, which allows making plots composed entirely of text characters. We can try it out by generating 10 thousand normally distributed random numbers, and making a histogram.

In [22]:
histogram(randn(10_000))

                ┌                                        ┐ 
   [-4.5, -4.0) ┤ 1                                        
   [-4.0, -3.5) ┤ 2                                        
   [-3.5, -3.0) ┤ 12                                       
   [-3.0, -2.5) ┤▇ 53                                      
   [-2.5, -2.0) ┤▇▇▇ 180                                   
   [-2.0, -1.5) ┤▇▇▇▇▇▇▇ 421                               
   [-1.5, -1.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 957                     
   [-1.0, -0.5) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 1419            
   [-0.5,  0.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 1906    
   [ 0.0,  0.5) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 1959   
   [ 0.5,  1.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 1523           
   [ 1.0,  1.5) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 875                       
   [ 1.5,  2.0) ┤▇▇▇▇▇▇▇▇ 456                              
   [ 2.0,  2.5) ┤▇▇▇ 170                                   
   [ 2.5,  3.0) ┤▇ 53                                      
   [ 3.0,  3.5) ┤ 12                    

Note that packages can rely on binaries. This is integrated into the design of Pkg, to provide a smooth installation experience. So, for example, people installing the Julia wrapper for the GDAL library, will automatically download a compatible GDAL installation compiled for their system. This is not installed globally, so will not interfere with the rest of the system. See https://binarybuilder.org/ to learn more about how this system works.

## Working with environments

When collaborating on a project, you want it to work the same on everyones computer. However, the default environment probably looks different for everybody, depending on the packages they have installed. This can mean not everyone has the same version of packages your project depends on. Not great for reproducible science!

Luckily Julia's package manager has the concept of environments. They can be created using `]activate path`, where path is the directory where the project is located, so `]activate .` creates an environment in the current directory.

In [40]:
]activate MyJuliaProject

 Activating new environment at `D:\repo\julia\julia-pizza-course\MyJuliaProject\Project.toml`


Now you can add packages that your project relies on. Note that if the same version of the package is already installed on your computer, for instance in another environment, it will use the same copy, to save space.

In [44]:
]add Example

  Resolving package versions...
No Changes to `D:\repo\julia\julia-pizza-course\MyJuliaProject\Project.toml`
No Changes to `D:\repo\julia\julia-pizza-course\MyJuliaProject\Manifest.toml`


To see the packages added to your current environment, use `]status`.

In [46]:
]status

Status `D:\repo\julia\julia-pizza-course\MyJuliaProject\Project.toml`
  [336ed68f] CSV v0.6.2
  [7876af07] Example v0.5.3


The state of your environment is captured in two files, the `Project.toml` and `Manifest.toml` files. We can print out their contents below.

In [54]:
println(read("MyJuliaProject/Project.toml", String))

[deps]
CSV = "336ed68f-0bac-5ca0-87d4-7b16caf5d00b"
Example = "7876af07-990d-54b4-ab0e-23690620f79a"



In [55]:
println(read("MyJuliaProject/Manifest.toml", String))

# This file is machine-generated - editing it directly is not advised

[[Base64]]
uuid = "2a0f44e3-6c83-55bd-87e4-b1978d98bd5f"

[[CSV]]
deps = ["CategoricalArrays", "DataFrames", "Dates", "FilePathsBase", "Mmap", "Parsers", "PooledArrays", "Tables", "Unicode", "WeakRefStrings"]
git-tree-sha1 = "52a8e60c7822f53d57e4403b7f2811e7e1bdd32b"
uuid = "336ed68f-0bac-5ca0-87d4-7b16caf5d00b"
version = "0.6.2"

[[CategoricalArrays]]
deps = ["DataAPI", "Future", "JSON", "Missings", "Printf", "Statistics", "Unicode"]
git-tree-sha1 = "a6c17353ee38ddab30e73dcfaa1107752de724ec"
uuid = "324d7699-5711-5eae-9e2f-1d82baa6b597"
version = "0.8.1"

[[Compat]]
deps = ["Base64", "Dates", "DelimitedFiles", "Distributed", "InteractiveUtils", "LibGit2", "Libdl", "LinearAlgebra", "Markdown", "Mmap", "Pkg", "Printf", "REPL", "Random", "SHA", "Serialization", "SharedArrays", "Sockets", "SparseArrays", "Statistics", "Test", "UUIDs", "Unicode"]
git-tree-sha1 = "48c20c43e157c6eab6cf88326504ec042b05e456"
uuid = "34da218

In short, the `Project.toml` file lists the dependencies under a `[deps]` section. It also supports other entries that can be added manually, for instance to add metadata or specify compatibility. The `Manifest.toml` specifies exactly which versions of both direct and indirect dependencies are used. If you share a `Manifest.toml` with someone, and someone uses `]instantiate` on it, they will be able to run your project with exactly the same dependencies.

More information can be found in these chapters of the documentation:
- https://julialang.github.io/Pkg.jl/v1/environments/
- https://julialang.github.io/Pkg.jl/v1/toml-files/
- https://julialang.github.io/Pkg.jl/v1/compatibility/